In [15]:
# Install necessary libraries
!pip install pandas nltk
!pip install ipywidgets
!pip install spotipy

# Importing libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import numpy as np
import string
from IPython.display import display, Markdown
import ipywidgets as widgets

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("Setup complete. Ready to load data.")

^C
ERROR: Operation cancelled by user
Setup complete. Ready to load data.


[nltk_data] Downloading package punkt to /Users/anita/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/anita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/anita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/anita/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


##### Clean Data

In [ ]:
# Import the Spotipy library
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initialize Spotipy with Spotify Developer Credentials
client_id = '1075d2afec3d414ca02ec7002f1d5576'  # Replace with your actual Spotify Client ID
client_secret = 'bb1fd201730b484983d952ecbd344b33'  # Replace with your actual Spotify Client Secret

credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=credentials)

In [ ]:
# Load data from a CSV file
data = pd.read_csv('/Users/anita/Documents/NLPSongs-/billboard_lyrics_1964-2015.csv', encoding='ISO-8859-1')
print("Data loaded successfully!")
print("Sample data:")
data # Display the first few rows of the dataset to verify it's loaded correctly

Data loaded successfully!
Sample data:


,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0
...,...,...,...,...,...,...
5095,96,el perdon,nicky jam and enrique iglesias,2015,enrique iglesias dime si es verdad me dijeron ...,3.0
5096,97,she knows,neyo featuring juicy j,2015,NaN,NaN
5097,98,night changes,one direction,2015,going out tonight changes into something red ...,1.0
5098,99,back to back,drake,2015,oh man oh man oh man not againyeah i learned ...,1.0


In [ ]:
null_values = data.isnull().sum()
print(null_values)

Rank        0
Song        0
Artist      0
Year        0
Lyrics    187
Source    187
dtype: int64


In [ ]:
#Remove null values
data = data.dropna()

In [ ]:
duplicate_rows = data.duplicated().sum()
print(duplicate_rows)

0


#### Preprocess

##### Tokenization and Lemmatization 

In [ ]:
def tokenize_and_lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

data['Preprocessed_data'] = data['Lyrics'].apply(tokenize_and_lemmatize)
print("Tokenization and lemmatization complete!")
print("Sample data:")
data['Preprocessed_data']

Tokenization and lemmatization complete!
Sample data:


/var/folders/hf/rxmfqb811qxdyqmz1rmf070c0000gn/T/ipykernel_57273/4038878403.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Preprocessed_data'] = data['Lyrics'].apply(tokenize_and_lemmatize)


0       sam the sham miscellaneous wooly bully wooly b...
1       sugar pie honey bunch you know that i love you...
2                                                        
3       when i woke up this morning you were on my min...
4       you never close your eye anymore when i kiss y...
                              ...                        
5094    my face above the water my foot cant touch the...
5095    enrique iglesias dime si e verdad me dijeron q...
5097    going out tonight change into something red he...
5098    oh man oh man oh man not againyeah i learned t...
5099    i want you to breathe me in let me be your air...
Name: Preprocessed_data, Length: 4913, dtype: object

In [ ]:
data = data.dropna(subset=['Preprocessed_data'])

# Remove rows with empty lyrics or that contain only one word
data = data[data['Preprocessed_data'].str.split().str.len() > 1]

#### Spotify Connection

In [17]:
def search_spotify(song_title, artist):
    query = f"track:{song_title} artist:{artist}"
    results = spotify.search(q=query, type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'preview_url': track['preview_url'],
            'spotify_url': track['external_urls']['spotify']
        }
    else:
        return "No results found on Spotify."

In [19]:
def retrieve_song_and_search_spotify(lyrics_input):
    processed_input = tokenize_and_lemmatize(lyrics_input)
    match = data[data['processed_lyrics'].str.contains(processed_input, na=False)]
    if not match.empty:
        song_title = match.iloc[0]['title']
        artist = match.iloc[0]['artist']
        spotify_info = search_spotify(song_title, artist)
        display_spotify_result(spotify_info)  # Using the new display function
    else:
        display(Markdown("**No matching song found.**"))

In [20]:
from IPython.display import display, HTML, Audio

def display_spotify_result(spotify_info):
    if isinstance(spotify_info, dict):
        html_content = f"<h3>{spotify_info['name']} by {spotify_info['artist']}</h3>"
        html_content += f"<p><strong>Album:</strong> {spotify_info['album']}</p>"
        html_content += f"<a href='{spotify_info['spotify_url']}' target='_blank'>Listen on Spotify</a>"
        if spotify_info['preview_url']:
            display(HTML(html_content))
            display(Audio(spotify_info['preview_url'], autoplay=False))
        else:
            display(HTML(html_content))
    else:
        print(spotify_info)

#### Search

In [21]:
import ipywidgets as widgets
from IPython.display import display

# Input for lyrics
lyrics_input = widgets.Text(
    value='',
    placeholder='Type lyrics here...',
    description='Lyrics:',
    disabled=False
)

# Button to trigger the search
search_button = widgets.Button(
    description='Search',
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    icon='search'  # FontAwesome icon name (without 'fa-')
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        retrieve_song_and_search_spotify(lyrics_input.value)

search_button.on_click(on_button_clicked)

display(lyrics_input, search_button, output)


Text(value='', description='Lyrics:', placeholder='Type lyrics here...')

Button(button_style='info', description='Search', icon='search', style=ButtonStyle())

Output()

we need more preprocessing, show metrics of the model, try and remove duplicates from the dataset in order to get more nuanced results, look to improve performance in every way possible, refine the data so that it is all fully in English